In [6]:
# import numpy as np
import mrcfile
from utils import crop_and_split_array

In [8]:
data = mrcfile.read('./20120923_Hylemonella_10003_full.rec')
cube_size = (64, 64, 64)
crop_and_split_array(data, cube_size, tile_path='../../images/3d_tiles/')


Saved all chunks as .npy files in ../../images/3d_tiles/


In [8]:
import gc
del data,chunks

In [12]:
import os
import ast
def recombine_chunks(og_rec_file,chunk_folder):
    """
    The following function reads in  orignal rec file to create an empty 3D array 
    then it reads in each of the chunk files and places them in the correct location
    based on the d,w,h coordinates in the file name. 
    """
    # Read in the original rec file to get the shape
    with mrcfile.open(og_rec_file) as mrc:
        # Use crop_3d to get the shape of the cropped array
        shape = crop_3d(mrc.data, (64, 64, 64)).shape

    # Create an empty 3D array
    recombined = np.zeros(shape)
    # Get a list of all the chunk files
    chunk_files = [f for f in os.listdir(chunk_folder) if f.endswith('.npy')]

    # Iterate through each file and place it in the correct location
    for file in chunk_files:
        # Get the x,y,z coordinates from the file name
        d,w,h = ast.literal_eval(file.split('_')[2].split('.npy')[0])
        d,w,h = int(d),int(w),int(h)
        # Load the chunk file
        chunk = np.load(f'{chunk_folder}{file}')
        # Place the chunk in the correct location
        cs= chunk.shape
        recombined[d*cs[0]:(d+1)*cs[0],w*cs[1]:(w+1)*cs[1],h*cs[2]:(h+1)*cs[2]] = chunk
    return recombined
file_path = './20120923_Hylemonella_10003_full.rec'
new_rec = recombine_chunks(file_path,'../../images/3d_tiles/')
np.save(f'{file_path[:-4]}_RECOMBINED',new_rec)
# recombine_chunks('./20120923_Hylemonella_10003_full.rec','../../images/3d_tiles/')


### Experiment with 3D plots

In [ ]:
def normalize_image(image):
    return cv2.equalizeHist(np.uint8(image))

In [ ]:
# Import data
import time
import numpy as np
import mrcfile
from skimage import io
import cv2

# vol = io.imread("https://s3.amazonaws.com/assets.datacamp.com/blog_assets/attention-mri.tif")
file_name = '20120923_Hylemonella_10003_full.rec'
vol = mrcfile.read(file_name)
vol = np.apply_along_axis(normalize_image, axis=0, arr=vol).squeeze(axis=1) 
print(type(vol))
volume = vol.T
r, c = volume[0].shape

print('volume shape: ',volume.shape)
# Define frames
import plotly.graph_objects as go
nb_frames = 300

fig = go.Figure(frames=[go.Frame(data=go.Surface(
    z=(6.7 - k * 0.1) * np.ones((r, c)),
    surfacecolor=np.flipud(volume[nb_frames - 1 - k]),
    cmin=0, cmax=200
    ),
    name=str(k) # you need to name the frame for the animation to behave properly
    )
    for k in range(nb_frames)])

# Add data to be displayed before animation starts
fig.add_trace(go.Surface(
    z=6.7 * np.ones((r, c)),
    surfacecolor=np.flipud(volume[299]),
    colorscale='Gray',
    cmin=0, cmax=200,
    colorbar=dict(thickness=20, ticklen=4)
    ))


def frame_args(duration):
    return {
            "frame": {"duration": duration},
            "mode": "immediate",
            "fromcurrent": True,
            "transition": {"duration": duration, "easing": "linear"},
        }

sliders = [
            {
                "pad": {"b": 10, "t": 60},
                "len": 0.9,
                "x": 0.1,
                "y": 0,
                "steps": [
                    {
                        "args": [[f.name], frame_args(0)],
                        "label": str(k),
                        "method": "animate",
                    }
                    for k, f in enumerate(fig.frames)
                ],
            }
        ]

# Layout
fig.update_layout(
         title='Slices in volumetric data',
         width=600,
         height=600,
         scene=dict(
                    zaxis=dict(range=[-0.1, 6.8], autorange=False),
                    aspectratio=dict(x=1, y=1, z=1),
                    ),
         updatemenus = [
            {
                "buttons": [
                    {
                        "args": [None, frame_args(50)],
                        "label": "&#9654;", # play symbol
                        "method": "animate",
                    },
                    {
                        "args": [[None], frame_args(0)],
                        "label": "&#9724;", # pause symbol
                        "method": "animate",
                    },
                ],
                "direction": "left",
                "pad": {"r": 10, "t": 70},
                "type": "buttons",
                "x": 0.1,
                "y": 0,
            }
         ],
         sliders=sliders
)

fig.show()

I probably want to think about saving each of these tiles/cubes into .npy files and save them with their coordinates in their name and the og file they came from.
One of the next steps I want to take is to use the plotly 3D (MRI) grapher to plot both the orignal 3D file and then on of the cubed versions too.